In [ ]:
import psycopg2
from google.colab import userdata

DB_URL = userdata.get('NEONDB')

try:
    conn = psycopg2.connect(DB_URL)
    cur = conn.cursor()

    cur.execute("""
    DROP TABLE IF EXISTS salaries, tasks, user_roles, roles, users, managers, departments, projects CASCADE;
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS projects (
        id SERIAL PRIMARY KEY,
        name TEXT NOT NULL,
        budget INT
    );
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS departments (
        id SERIAL PRIMARY KEY,
        name TEXT NOT NULL,
        location TEXT
    );
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id SERIAL PRIMARY KEY,
        name TEXT NOT NULL,
        age INT,
        department_id INT,
        manager_id INT,
        FOREIGN KEY (department_id)
            REFERENCES departments(id)
            ON DELETE SET NULL,
        FOREIGN KEY (manager_id)
            REFERENCES users(id)
            ON DELETE SET NULL
    );
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS roles (
        id SERIAL PRIMARY KEY,
        name TEXT NOT NULL UNIQUE
    );
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS user_roles (
        user_id INT,
        role_id INT,
        PRIMARY KEY (user_id, role_id),
        FOREIGN KEY (user_id) REFERENCES users(id) ON DELETE CASCADE,
        FOREIGN KEY (role_id) REFERENCES roles(id) ON DELETE CASCADE
    );
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS tasks (
        id SERIAL PRIMARY KEY,
        title TEXT NOT NULL,
        status TEXT CHECK (status IN ('todo', 'in_progress', 'done')),
        project_id INT,
        assigned_user_id INT,
        FOREIGN KEY (project_id) REFERENCES projects(id) ON DELETE CASCADE,
        FOREIGN KEY (assigned_user_id) REFERENCES users(id) ON DELETE SET NULL
    );
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS salaries (
        id SERIAL PRIMARY KEY,
        user_id INT,
        amount INT NOT NULL,
        valid_from DATE NOT NULL,
        valid_to DATE,
        FOREIGN KEY (user_id) REFERENCES users(id) ON DELETE CASCADE
    );
    """)

    cur.executemany("""
    INSERT INTO projects (name, budget)
    VALUES (%s, %s);
    """, [
        ('Projekt Alfa', 50000),
        ('Projekt Beta', 75000),
        ('Projekt Gamma', 120000),
        ('Projekt Delta', 30000)
    ])

    cur.executemany("""
    INSERT INTO departments (name, location)
    VALUES (%s, %s);
    """, [
        ('Marketing', 'Warszawa'),
        ('IT', 'Kraków'),
        ('Finanse', 'Londyn'),
        ('Kadry', 'Berlin'),
        ('Operacje', 'Tokio')
    ])

    cur.executemany("""
    INSERT INTO users (name, age, department_id, manager_id)
    VALUES (%s, %s, %s, %s);
    """, [
        ('Jan Kowalski', 45, 1, None),
        ('Maria Nowak', 38, 2, None),
        ('Robert Wiśniewski', 50, 3, None),
        ('Anna Kaczmarek', 32, 4, None),
        ('Alicja', 30, 1, 1),
        ('Bartosz', 25, 2, 2),
        ('Karol', 35, 3, 3),
        ('Daria', 28, 4, 4),
        ('Ewelina', 42, 5, 1)
    ])

    cur.executemany("""
    INSERT INTO roles (name)
    VALUES (%s);
    """, [
        ('Manager',),
        ('Programista',),
        ('Analityk',),
        ('Specjalista HR',),
        ('Lider zespołu',)
    ])

    cur.executemany("""
    INSERT INTO user_roles (user_id, role_id)
    VALUES (%s, %s);
    """, [
        (1, 1),
        (2, 1),
        (3, 1),
        (4, 1),
        (5, 2),
        (6, 2),
        (7, 3),
        (8, 4),
        (9, 5)
    ])

    cur.executemany("""
    INSERT INTO tasks (title, status, project_id, assigned_user_id)
    VALUES (%s, %s, %s, %s);
    """, [
        ('Zarządzanie projektem Alfa', 'in_progress', 1, 1),
        ('Projekt bazy danych', 'done', 1, 5),
        ('Implementacja API', 'in_progress', 1, 6),
        ('Przygotowanie raportu finansowego', 'todo', 3, 7),
        ('Wdrożenie nowego pracownika', 'done', 4, 8)
    ])

    cur.executemany("""
    INSERT INTO salaries (user_id, amount, valid_from, valid_to)
    VALUES (%s, %s, %s, %s);
    """, [
        (1, 12000, '2023-01-01', None),
        (2, 9500, '2023-01-01', None),
        (3, 14000, '2023-01-01', None),
        (4, 8700, '2023-01-01', None),
        (5, 4800, '2023-01-01', '2023-12-31'),
        (5, 5000, '2024-01-01', None),
        (6, 4200, '2024-01-01', None),
        (7, 6200, '2024-01-01', None),
        (8, 4800, '2024-01-01', None),
        (9, 7000, '2024-01-01', None)
    ])

    conn.commit()
    print("Struktura bazy została zaktualizowana i wypełniona danymi.")

    cur.close()
    conn.close()

except Exception as e:
    print("Błąd:", e)